Importar librerías

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
import os

Acceder a las bases de datos usando config.yml

In [2]:
NOTEBOOK_DIR = os.getcwd()
ROOT_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, "..", ".."))
config_path = os.path.join(ROOT_DIR, "config.yml")

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
co_sa = create_engine(url_co)
etl_co = create_engine(url_etl)

Guardar las tablas para acceder a la infromación

In [165]:
beneficiario = pd.read_sql_table("beneficiario", co_sa)
cotizante = pd.read_sql_table("cotizante", co_sa)
cotizante_beneficiario = pd.read_sql_table("cotizante_beneficiario", co_sa)

# citas_generales = pd.read_sql_table("citas_generales", co_sa)
# drogueria = pd.read_sql_table("drogueria", co_sa)
# empresa = pd.read_sql_table("empresa", co_sa)
# empresa_cotizante = pd.read_sql_table("empresa_cotizante", co_sa)
# formulas_medicas = pd.read_sql_table("formulas_medicas", co_sa)
# hospitalizaciones = pd.read_sql_table("hospitalizaciones", co_sa)
# ips = pd.read_sql_table("ips", co_sa)
# ips_drogueria = pd.read_sql_table("ips_drogueria", co_sa)
# medico = pd.read_sql_table("medico", co_sa)
# pagos = pd.read_sql_table("pagos", co_sa)
# preexistencias = pd.read_sql_table("preexistencias", co_sa)
# remisiones = pd.read_sql_table("remisiones", co_sa)
# retiros = pd.read_sql_table("retiros", co_sa)
# servicios_pos = pd.read_sql_table("servicios_pos", co_sa)
# urgencias = pd.read_sql_table("urgencias", co_sa)

#También se puede hacer un select dentro del read si no se necesitan todas las columnas

In [166]:
#También se puede así
tablas = [
    "beneficiario", "citas_generales", "cotizante", "cotizante_beneficiario",
    "drogueria", "empresa", "empresa_cotizante", "formulas_medicas",
    "hospitalizaciones", "ips", "ips_drogueria", "medico", "pagos",
    "preexistencias", "remisiones", "retiros", "servicios_pos", "urgencias"
]

dfs = {tabla: pd.read_sql_table(tabla, co_sa) for tabla in tablas}

#Y se lee así
# dfs["beneficiario"]
# dfs["medico"]

Revisar si hay vacíos y si las columnas con opciones tienen respuestas por fuera de las opciones

In [167]:
print("Info beneficiarios: ")
beneficiario.info()
print("Sexo beneficiarios: ", beneficiario.sexo.unique())

Info beneficiarios: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3316 entries, 0 to 3315
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   tipo_identificacion  3316 non-null   object        
 1   id_beneficiario      3316 non-null   object        
 2   parentesco           3316 non-null   object        
 3   nombre               3316 non-null   object        
 4   fecha_nacimiento     3316 non-null   datetime64[ns]
 5   sexo                 3316 non-null   object        
 6   estado_civil         3316 non-null   object        
 7   tipo_discapacidad    3316 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 207.4+ KB
Sexo beneficiarios:  ['F' 'M']


In [168]:
beneficiario.describe(include='all')

,tipo_identificacion,id_beneficiario,parentesco,nombre,fecha_nacimiento,sexo,estado_civil,tipo_discapacidad
count,3316,3316,3316,3316,3316,3316,3316,3316
unique,2,3316,11,3316,NaN,2,5,4
top,Cedula,1135438563850,Hijo,Maribel Medrano Pardo,NaN,M,Soltero,Ninguna
freq,2549,1,643,1,NaN,1747,1186,2620
mean,NaN,NaN,NaN,NaN,1964-03-31 07:45:05.428226752,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1904-01-08 00:00:00,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,1941-02-01 00:00:00,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,1965-06-19 00:00:00,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1989-08-26 06:00:00,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,2000-12-26 00:00:00,NaN,NaN,NaN


Vamos a crear la tabla Dim_persona de la olap, que es una combinación de Beneficiario y Cotizante de la oltp. Para esto hay que hacer algunas trasnformaciones.

| Beneficiario            | Cotizante              | Dim_persona              |
|--------------------------|------------------------|--------------------------|
| tipo_identificacion      | se crea campo con valor constante **cedula** | tipo_documento |
| nombre                   | nombre                 | nombre                   |
| fecha_nacimiento         | fecha_nacimiento       | fecha_nacimiento         |
| sexo                     | sexo                   | sexo                     |
| estado_civil             | estado_civil           | estado_civil             |
| tipo_discapacidad        | tipo_discapacidad      | tipo_discapacidad        |
| se crea campo valor constante **beneficiario** | se crea campo valor constante **cotizante** | tipo_usuario |
|se crea campo valor extraido de cotizante_beneficiario|se crea campo valor igual al campo cédula|grupo_familiar                          |
| beneficiario_id          | cedula                 | numero_identificacion    |


In [169]:
print("beneficiario:", beneficiario.columns)
print("cotizante:" ,cotizante.columns)

beneficiario: Index(['tipo_identificacion', 'id_beneficiario', 'parentesco', 'nombre',
       'fecha_nacimiento', 'sexo', 'estado_civil', 'tipo_discapacidad'],
      dtype='object')
cotizante: Index(['cedula', 'nombre', 'tipo_cotizante', 'direccion', 'estado_civil',
       'sexo', 'fecha_nacimiento', 'nivel_escolaridad', 'estracto',
       'proviene_otra_eps', 'salario_base', 'fecha_afiliacion',
       'tipo_discapacidad', 'id_ips'],
      dtype='object')


In [170]:
# Se cambia el nombre de las columnas que sean diferentes
#Se pone inplace= True o se iguala a la variable
beneficiario.rename(columns={'tipo_identificacion':'tipo_documento',\
  'id_beneficiario':'numero_identificacion'}, inplace= True)

cotizante = cotizante.rename(columns={'cedula':'numero_identificacion'})

#Se crean lo campos que no existen en la oltp pero sí en la olap
beneficiario['tipo_usuario'] = 'beneficiario'
cotizante['tipo_usuario'] = 'cotizante'
cotizante['tipo_documento'] = 'cedula'

#Borramos las columnas que no se usan en la olap
beneficiario = beneficiario.drop(columns=['parentesco'])
cotizante = cotizante.drop(columns=['tipo_cotizante'])
cotizante = cotizante.drop(columns=['direccion'])
cotizante = cotizante.drop(columns=['nivel_escolaridad'])
cotizante = cotizante.drop(columns=['estracto'])
cotizante = cotizante.drop(columns=['proviene_otra_eps'])
cotizante = cotizante.drop(columns=['salario_base'])
cotizante = cotizante.drop(columns=['fecha_afiliacion'])
cotizante = cotizante.drop(columns=['id_ips'])
print("beneficiario:", beneficiario.columns)
print("cotizante:" ,cotizante.columns)

beneficiario: Index(['tipo_documento', 'numero_identificacion', 'nombre', 'fecha_nacimiento',
       'sexo', 'estado_civil', 'tipo_discapacidad', 'tipo_usuario'],
      dtype='object')
cotizante: Index(['numero_identificacion', 'nombre', 'estado_civil', 'sexo',
       'fecha_nacimiento', 'tipo_discapacidad', 'tipo_usuario',
       'tipo_documento'],
      dtype='object')


Ahora vamos a añadir el campo grupo_familiar usando cotizante_beneficiario, que relaciona las llaves. Se usa merge (join)

In [171]:
print("cotizante_beneficiario:", cotizante_beneficiario.columns)

cotizante_beneficiario: Index(['cotizante', 'beneficiario'], dtype='object')


In [172]:
beneficiario = beneficiario.merge(cotizante_beneficiario, left_on= 'numero_identificacion', right_on= 'beneficiario', how='left')
beneficiario = beneficiario.drop(columns=['beneficiario'])
beneficiario.rename(columns={'cotizante':'grupo_familiar'}, inplace= True)

cotizante['grupo_familiar'] = cotizante['numero_identificacion']

print("beneficiario:", beneficiario.columns)
print("cotizante:" ,cotizante.columns)

beneficiario: Index(['tipo_documento', 'numero_identificacion', 'nombre', 'fecha_nacimiento',
       'sexo', 'estado_civil', 'tipo_discapacidad', 'tipo_usuario',
       'grupo_familiar'],
      dtype='object')
cotizante: Index(['numero_identificacion', 'nombre', 'estado_civil', 'sexo',
       'fecha_nacimiento', 'tipo_discapacidad', 'tipo_usuario',
       'tipo_documento', 'grupo_familiar'],
      dtype='object')


Ahora sí vamos a crear la tabla dim_persona usando otro concat

In [173]:
dim_persona = pd.concat([beneficiario,cotizante])

In [174]:
beneficiario

,tipo_documento,numero_identificacion,nombre,fecha_nacimiento,sexo,estado_civil,tipo_discapacidad,tipo_usuario,grupo_familiar
0,Cedula,1705221112240,Deidamia Bernal Tello,1954-10-08,F,Soltero,Ninguna,beneficiario,1170522
1,Tarjeta Identidad,1705221115600,Nazaret Dueñas Dueñas,1997-10-16,F,Soltero,Ninguna,beneficiario,2170522
2,Cedula,1705221116830,Afra Acosta Heredia,1975-08-21,F,Union Libre,Ninguna,beneficiario,2170522
3,Cedula,1705221117960,Tomás Dueñas Delatorre,1967-07-24,M,Union Libre,Ninguna,beneficiario,2170522
4,Cedula,1705221123050,Celeste Díez Calderón,1949-08-05,F,Casado,Ninguna,beneficiario,5170522
...,...,...,...,...,...,...,...,...,...
3311,Cedula,1135438558010,Jordana Lopez Nieto,1952-07-02,F,Casado,Ninguna,beneficiario,998113543
3312,Cedula,1135438560540,Fedro Medrano Crespo,1944-08-23,M,Soltero,Ninguna,beneficiario,999113543
3313,Cedula,1135438561640,Baudilia Alfaro Lira,1957-12-27,F,Divorciado,Mental,beneficiario,999113543
3314,Tarjeta Identidad,1135438562650,Baltasar Medrano Alfaro,1995-01-23,M,Soltero,Ninguna,beneficiario,999113543


In [175]:
cotizante

,numero_identificacion,nombre,estado_civil,sexo,fecha_nacimiento,tipo_discapacidad,tipo_usuario,tipo_documento,grupo_familiar
0,932170522,Eleodora Tapia Lopez,Viudo,F,1983-09-05,Ninguna,cotizante,cedula,932170522
1,934170522,Belisaria Crespo Jerez,Union Libre,F,1990-12-23,Ninguna,cotizante,cedula,934170522
2,936170522,Gerardo Villanueva Pinto,Casado,M,1983-10-23,Ninguna,cotizante,cedula,936170522
3,937170522,Bienvenido Quintana Moreno,Viudo,M,1987-02-16,Ninguna,cotizante,cedula,937170522
4,941170522,Celestino Medrano Moreno,Union Libre,M,1990-01-24,Ninguna,cotizante,cedula,941170522
...,...,...,...,...,...,...,...,...,...
1095,993113543,Elvis Caballero Ulloa,Soltero,M,1954-04-26,Ninguna,cotizante,cedula,993113543
1096,994113543,Beltrán Vallejo Rosales,Divorciado,M,1962-06-25,Ninguna,cotizante,cedula,994113543
1097,995113543,Elina Carrasco González,Union Libre,F,1974-08-24,Ninguna,cotizante,cedula,995113543
1098,996113543,Adria Trujillo Pinto,Casado,F,1945-11-16,Ninguna,cotizante,cedula,996113543


3.316 + 1.100 = 4.416 filas

In [176]:
dim_persona

,tipo_documento,numero_identificacion,nombre,fecha_nacimiento,sexo,estado_civil,tipo_discapacidad,tipo_usuario,grupo_familiar
0,Cedula,1705221112240,Deidamia Bernal Tello,1954-10-08,F,Soltero,Ninguna,beneficiario,1170522
1,Tarjeta Identidad,1705221115600,Nazaret Dueñas Dueñas,1997-10-16,F,Soltero,Ninguna,beneficiario,2170522
2,Cedula,1705221116830,Afra Acosta Heredia,1975-08-21,F,Union Libre,Ninguna,beneficiario,2170522
3,Cedula,1705221117960,Tomás Dueñas Delatorre,1967-07-24,M,Union Libre,Ninguna,beneficiario,2170522
4,Cedula,1705221123050,Celeste Díez Calderón,1949-08-05,F,Casado,Ninguna,beneficiario,5170522
...,...,...,...,...,...,...,...,...,...
1095,cedula,993113543,Elvis Caballero Ulloa,1954-04-26,M,Soltero,Ninguna,cotizante,993113543
1096,cedula,994113543,Beltrán Vallejo Rosales,1962-06-25,M,Divorciado,Ninguna,cotizante,994113543
1097,cedula,995113543,Elina Carrasco González,1974-08-24,F,Union Libre,Ninguna,cotizante,995113543
1098,cedula,996113543,Adria Trujillo Pinto,1945-11-16,F,Casado,Ninguna,cotizante,996113543


In [177]:
#No puede tener el nombre de una tabla ya creada
dim_persona.to_sql('dim_persona_ELT', etl_co, if_exists='replace', index_label='key_dim_persona')

416